In [1]:
import torch
import os


from ultralytics import YOLO


In [2]:

cuda_available = torch.cuda.is_available()

if cuda_available:

  print("CUDA is available")
  !nvidia-smi

else:

  message = """
    WARNING: In order to train the model, it is advisable to use GPU.
    Change runtime type to GPU from:
      menu Runtime -> Change runtime type -> Hardware accelerator -> GPU.
      And run all the cells again.
  """
  print(message)


CUDA is available
Tue Nov 18 19:20:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           On  |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             50W /  300W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------

In [3]:
print("Torch:", torch.__version__, "CUDA runtime:", torch.version.cuda)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print("Capability:", torch.cuda.get_device_capability(0))

Torch: 2.9.1+cu128 CUDA runtime: 12.8
Tesla V100-SXM2-16GB
Capability: (7, 0)


In [4]:

def serialized_model_file(
    checkpoint="best",
    use_run="train",
):
  """
    Returns the serialized file path.
  """
  return f"runs/detect/{use_run}/weights/{checkpoint}.pt"


In [ ]:

EPOCHS = 10
BATCH = 4
IMG_SIZE = 640
FALLBACK = "yolo11s.pt"

def train(
    data,
    use_run="train",
    fallback=FALLBACK,
    epochs=EPOCHS,
    augment=True,
  ):

  cuda_available = torch.cuda.is_available()

  if not cuda_available:
    print("Warning CUDA is not available, train would take too much time.")
    # return

  if cuda_available:
    torch.cuda.empty_cache()

  model_file = serialized_model_file("last", use_run)

  if os.path.exists(model_file):
    resume_training = True
    use_model = model_file
    
  else:
    resume_training = False
    use_model = fallback

  model = YOLO(
      use_model
  )

  model.train(
      data = data,
      resume = resume_training,
      epochs = epochs,
      device = 'cuda',
      optimizer="AdamW",
      # lr0=0.0001,
      imgsz = IMG_SIZE,
      batch = BATCH,
      augment = augment,
      amp = False
  )

In [6]:
train(
    data="data/tracking/YOLO/tracker.yaml",
    use_run=None,
    augment=False
)


Ultralytics 8.3.228 🚀 Python-3.13.9 torch-2.9.1+cu128 CUDA:0 (Tesla V100-SXM2-16GB, 16144MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/tracking/YOLO/tracker.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plot

  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1    990976  ultralytics.nn.modules.block.C2PSA           [512, 512, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    443776  ultralytics.nn.modules.block.C3k2            [768,